In [1]:
pip install requests pandas tqdm


Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import pandas as pd
from tqdm import tqdm
import time

API_KEY = '35250d55ae36d4d2136a5c928021acc1'

# Get a list of popular movies (multiple pages)
def get_popular_movies(api_key, total_pages=100):
    movies = []
    for page in tqdm(range(1, total_pages + 1), desc="Fetching movie list"):
        url = f'https://api.themoviedb.org/3/movie/popular'
        params = {'api_key': api_key, 'language': 'en-US', 'page': page}
        response = requests.get(url, params=params)
        if response.status_code != 200:
            break
        data = response.json()
        for movie in data['results']:
            movies.append((movie['id'], movie['title']))
        time.sleep(0.2)  # Be kind to the API
    return movies

# Get reviews for a single movie
def get_reviews(movie_id, api_key, max_pages=5):
    reviews = []
    for page in range(1, max_pages + 1):
        url = f'https://api.themoviedb.org/3/movie/{movie_id}/reviews'
        params = {'api_key': api_key, 'language': 'en-US', 'page': page}
        response = requests.get(url, params=params)
        if response.status_code != 200:
            break
        data = response.json()
        if not data.get('results'):
            break
        for r in data['results']:
            reviews.append({
                'movie_id': movie_id,
                'author': r['author'],
                'review': r['content'],
                'rating': r['author_details'].get('rating'),
            })
        time.sleep(0.2)
    return reviews

# Main script to scrape ~50,000 reviews
all_reviews = []
max_reviews = 50000
movies = get_popular_movies(API_KEY, total_pages=200)

print(f"\nTotal movies fetched: {len(movies)}")

for movie_id, title in tqdm(movies, desc="Scraping reviews"):
    reviews = get_reviews(movie_id, API_KEY, max_pages=10)
    for r in reviews:
        r['movie_title'] = title
    all_reviews.extend(reviews)
    if len(all_reviews) >= max_reviews:
        break

# Save dataset
df = pd.DataFrame(all_reviews)
df.drop_duplicates(subset='review', inplace=True)
df.to_csv('tmdb_reviews_dataset.csv', index=False)

print(f"\n✅ Collected {len(df)} unique reviews and saved to 'tmdb_reviews_dataset.csv'")


Fetching movie list: 100%|███████████████████████████████████████████████████████████| 200/200 [03:17<00:00,  1.01it/s]



Total movies fetched: 4000


Scraping reviews: 100%|██████████████████████████████████████████████████████████| 4000/4000 [1:18:18<00:00,  1.17s/it]



✅ Collected 8359 unique reviews and saved to 'tmdb_reviews_dataset.csv'


In [1]:
import requests
import pandas as pd
from tqdm import tqdm
import time
import os

API_KEY = '35250d55ae36d4d2136a5c928021acc1'

# ------------------------- MOVIE FETCHING -------------------------
def get_discover_movies(api_key, total_pages=300):
    movies = set()
    for page in tqdm(range(1, total_pages + 1), desc="Discovering movies"):
        url = 'https://api.themoviedb.org/3/discover/movie'
        params = {
            'api_key': api_key,
            'language': 'en-US',
            'sort_by': 'vote_count.desc',
            'vote_count.gte': 50,
            'page': page
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            break
        data = response.json()
        for movie in data['results']:
            movies.add((movie['id'], movie['title']))
        time.sleep(0.1)
    return list(movies)

def get_popular_movies(api_key, total_pages=100):
    movies = set()
    for page in tqdm(range(1, total_pages + 1), desc="Fetching popular movies"):
        url = 'https://api.themoviedb.org/3/movie/popular'
        params = {
            'api_key': api_key,
            'language': 'en-US',
            'page': page
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            break
        data = response.json()
        for movie in data['results']:
            movies.add((movie['id'], movie['title']))
        time.sleep(0.1)
    return list(movies)

# ------------------------- REVIEW FETCHING -------------------------
def get_reviews(movie_id, api_key, max_pages=20):
    reviews = []
    for page in range(1, max_pages + 1):
        url = f'https://api.themoviedb.org/3/movie/{movie_id}/reviews'
        params = {
            'api_key': api_key,
            'language': 'en-US',
            'page': page
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            break
        data = response.json()
        if not data.get('results'):
            break
        for r in data['results']:
            reviews.append({
                'movie_id': movie_id,
                'author': r['author'],
                'review': r['content'],
                'rating': r['author_details'].get('rating'),
            })
        time.sleep(0.1)
    return reviews

# ------------------------- MAIN SCRAPER -------------------------
def scrape_reviews(api_key, max_reviews=60000, save_every=2000):
    # Combine movies from multiple sources
    discover = get_discover_movies(api_key, total_pages=300)
    popular = get_popular_movies(api_key, total_pages=100)
    all_movies = list(set(discover + popular))

    print(f"\n🎬 Total unique movies found: {len(all_movies)}")

    all_reviews = []
    processed_movies = 0

    for movie_id, title in tqdm(all_movies, desc="Scraping reviews"):
        reviews = get_reviews(movie_id, api_key, max_pages=20)
        for r in reviews:
            r['movie_title'] = title
        all_reviews.extend(reviews)
        processed_movies += 1

        # Save progress every few thousand reviews
        if len(all_reviews) >= save_every and len(all_reviews) % save_every < len(reviews):
            df = pd.DataFrame(all_reviews)
            df.drop_duplicates(subset='review', inplace=True)
            df.to_csv(f'tmdb_reviews_partial_{len(df)}.csv', index=False)
            print(f"\n💾 Saved intermediate file: {len(df)} reviews")

        # Stop if max reached
        if len(all_reviews) >= max_reviews:
            break

    # Final save
    df = pd.DataFrame(all_reviews)
    df.drop_duplicates(subset='review', inplace=True)
    df.to_csv('tmdb_reviews_dataset2.csv', index=False)

    print(f"\n✅ Collected {len(df)} unique reviews and saved to 'tmdb_reviews_dataset2.csv'")

# ------------------------- RUN SCRIPT -------------------------
if __name__ == '__main__':
    scrape_reviews(API_KEY, max_reviews=60000, save_every=2000)


Fetching popular movies: 100%|███████████████████████████████████████████████████████| 100/100 [00:56<00:00,  1.78it/s]



🎬 Total unique movies found: 6564


Scraping reviews:  13%|███████▍                                                   | 833/6564 [12:06<1:25:04,  1.12it/s]


💾 Saved intermediate file: 1991 reviews


Scraping reviews:  27%|███████████████▍                                          | 1743/6564 [25:29<1:11:55,  1.12it/s]

...


💾 Saved intermediate file: 11908 reviews


Scraping reviews:  94%|██████████████████████████████████████████████████████▏   | 6138/6564 [1:29:30<07:34,  1.07s/it]


💾 Saved intermediate file: 13893 reviews


Scraping reviews: 100%|██████████████████████████████████████████████████████████| 6564/6564 [1:35:33<00:00,  1.14it/s]



✅ Collected 14864 unique reviews and saved to 'tmdb_reviews_dataset2.csv'


In [2]:
import pandas as pd 
df2=pd.read_csv("tmdb_reviews_dataset2.csv") 
df2.head()

,movie_id,author,review,rating,movie_title
0,8452,John Chard,If you really believe that then you should clo...,6.5,The 6th Day
1,8452,Gimly,_The 6th Day_ is a **great** title for a movie...,6.0,The 6th Day
2,17578,ohlalipop,The animation was something else. It looked so...,10.0,The Adventures of Tintin
3,17578,lmao7,Wow…they just took cg/motion/performance captu...,9.0,The Adventures of Tintin
4,351460,.,_Death Note_ - _★★★★_\r\n\r\nWhile different f...,8.0,Death Note


In [5]:
len(df2['movie_title'].unique())

4644

In [5]:
import pandas as pd 
df=pd.read_csv("tmdb_reviews_dataset.csv") 


In [6]:
df.shape

(8359, 5)

In [7]:
df.head()

,movie_id,author,review,rating,movie_title
0,1241436,CinemaSerf,A squad of American soldiers seemingly randoml...,7.0,Warfare
1,1241436,r96sk,<em>'Warfare'</em> ends up as expected: bleak ...,8.0,Warfare
2,1241436,MovieGuys,"""Warfare"" is as a simple statement on the real...",7.0,Warfare
3,1241436,Manuel São Bento,FULL SPOILER-FREE REVIEW @ https://movieswetex...,7.0,Warfare
4,950387,tmdb98094809,"Alright, buckle up, because I just saw a movie...",10.0,A Minecraft Movie


In [10]:
df['movie_id'].value_counts()

movie_id
299534     58
324857     53
24428      41
76338      39
284053     38
           ..
619         1
3563        1
10715       1
23048       1
1151039     1
Name: count, Length: 2183, dtype: int64

In [11]:
df.drop_duplicates()

,movie_id,author,review,rating,movie_title
0,1241436,CinemaSerf,A squad of American soldiers seemingly randoml...,7.0,Warfare
1,1241436,r96sk,<em>'Warfare'</em> ends up as expected: bleak ...,8.0,Warfare
2,1241436,MovieGuys,"""Warfare"" is as a simple statement on the real...",7.0,Warfare
3,1241436,Manuel São Bento,FULL SPOILER-FREE REVIEW @ https://movieswetex...,7.0,Warfare
4,950387,tmdb98094809,"Alright, buckle up, because I just saw a movie...",10.0,A Minecraft Movie
...,...,...,...,...,...
8354,181283,GenerationofSwine,"Despite Oldman's involvement, it lacks all the...",5.0,Child 44
8355,77174,John Chard,"Mom, tell the Zombie to stop saying stuff abou...",8.0,ParaNorman
8356,77174,Kamurai,"Really good watch, could watch again, and can ...",7.0,ParaNorman
8357,77174,CinemaSerf,"His parents think he's a bit bonkers, but ""Nor...",7.0,ParaNorman


In [12]:
df.shape

(8359, 5)

In [ ]:
import os
import csv
import time
import requests
from tqdm import tqdm

API_KEY = "35250d55ae36d4d2136a5c928021acc1"
SAVE_FILE = "tmdb_movie_reviews_large.csv"
MAX_MOVIE_PAGES = 1000    # 20 movies per page × 1000 = 20,000 movies
MAX_REVIEW_PAGES = 10     # Max pages of reviews to fetch per movie (adjust as needed)

def get_movie_ids(api_key, max_pages=MAX_MOVIE_PAGES):
    movie_ids = []
    for page in tqdm(range(1, max_pages + 1), desc="Fetching movie IDs"):
        url = f"https://api.themoviedb.org/3/discover/movie"
        params = {
            "api_key": api_key,
            "language": "en-US",
            "sort_by": "popularity.desc",
            "page": page
        }
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            for movie in data.get("results", []):
                movie_ids.append((movie["id"], movie["title"]))
        except Exception as e:
            print(f"Failed to fetch movies on page {page}: {e}")
            time.sleep(2)
            continue
    return movie_ids

def get_reviews(movie_id, movie_title, api_key, max_pages=MAX_REVIEW_PAGES):
    reviews = []
    for page in range(1, max_pages + 1):
        url = f"https://api.themoviedb.org/3/movie/{movie_id}/reviews"
        params = {
            "api_key": api_key,
            "language": "en-US",
            "page": page
        }
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            for item in data.get("results", []):
                reviews.append({
                    "movie_id": movie_id,
                    "movie_title": movie_title,
                    "author": item["author"],
                    "review": item["content"]
                })
            if page >= data.get("total_pages", 1):
                break
        except Exception as e:
            print(f"Error fetching reviews for movie {movie_id}: {e}")
            break
    return reviews

def save_reviews_csv(reviews, file_path):
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["movie_id", "movie_title", "author", "review"])
        if not file_exists:
            writer.writeheader()
        writer.writerows(reviews)

def get_already_scraped_ids(file_path):
    if not os.path.isfile(file_path):
        return set()
    scraped_ids = set()
    with open(file_path, mode="r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            scraped_ids.add(int(row["movie_id"]))
    return scraped_ids

def main():
    all_movies = get_movie_ids(API_KEY)
    scraped_ids = get_already_scraped_ids(SAVE_FILE)

    for movie_id, movie_title in tqdm(all_movies, desc="Scraping reviews"):
        if movie_id in scraped_ids:
            continue
        reviews = get_reviews(movie_id, movie_title, API_KEY)
        if reviews:
            save_reviews_csv(reviews, SAVE_FILE)
        time.sleep(0.5)  # Prevent rate-limiting

    print(f"\n✅ Scraping complete. Reviews saved to: {SAVE_FILE}")

if __name__ == "__main__":
    main()


In [ ]:
import os
import csv
import requests
from tqdm import tqdm
from langdetect import detect

API_KEY = "35250d55ae36d4d2136a5c928021acc1"
MOVIE_LIST_PAGES = 500  # Up to 10,000 movies (20 movies per page)
REVIEWS_PER_MOVIE_PAGES = 20  # Try to get max reviews per movie
CSV_FILE_NAME = "tmdb_english_movie_reviews.csv"

# Create the CSV file with headers if not exists
if not os.path.exists(CSV_FILE_NAME):
    with open(CSV_FILE_NAME, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=["movie_id", "movie_title", "review_id", "author", "content", "created_at"])
        writer.writeheader()

def get_movie_list(api_key, total_pages):
    movies = []
    for page in tqdm(range(1, total_pages + 1), desc="Fetching movie list"):
        url = f"https://api.themoviedb.org/3/discover/movie"
        params = {
            "api_key": api_key,
            "language": "en-US",
            "sort_by": "popularity.desc",
            "page": page
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            results = response.json().get("results", [])
            for movie in results:
                movies.append((movie["id"], movie["title"]))
        else:
            break
    return movies

def get_reviews(movie_id, movie_title, api_key, max_pages):
    reviews = []
    for page in range(1, max_pages + 1):
        url = f"https://api.themoviedb.org/3/movie/{movie_id}/reviews"
        params = {
            "api_key": api_key,
            "language": "en-US",
            "page": page
        }
        response = requests.get(url, params=params)
        if response.status_code != 200:
            break
        data = response.json().get("results", [])
        if not data:
            break
        for review in data:
            try:
                if detect(review["content"]) == "en":
                    reviews.append({
                        "movie_id": movie_id,
                        "movie_title": movie_title,
                        "review_id": review["id"],
                        "author": review["author"],
                        "content": review["content"],
                        "created_at": review["created_at"]
                    })
            except:
                continue
    return reviews

# Main scraper loop
movies = get_movie_list(API_KEY, MOVIE_LIST_PAGES)

for movie_id, movie_title in tqdm(movies, desc="Scraping reviews"):
    reviews = get_reviews(movie_id, movie_title, API_KEY, REVIEWS_PER_MOVIE_PAGES)
    if reviews:
        with open(CSV_FILE_NAME, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=["movie_id", "movie_title", "review_id", "author", "content", "created_at"])
            for review in reviews:
                writer.writerow(review)
